In [1]:
#pip3 install notebook ipywidgets plotly
#jupyter nbextension enable --py widgetsnbextension
#jupyter nbextension enable --py plotlywidget

In [2]:
import igraph as ig
import pandas as pd
import webbrowser
import json
import urllib.request
import chart_studio.plotly as py
import plotly.graph_objs as go
from ipywidgets import widgets

from utils import *

import sys
sys.path.append("..")
# from predict import *

This visualization is inspired from plotly officiel tutorials: https://plot.ly/python/v3/3d-network-graph/

In [3]:
# Load some data
data = []
req = urllib.request.Request("https://raw.githubusercontent.com/plotly/datasets/master/miserables.json")
opener = urllib.request.build_opener()
f = opener.open(req)
data = json.loads(f.read())

In [4]:
df_node = pd.DataFrame(data['nodes'])
df_node.drop(columns=['group'],axis=1,inplace=True)
df_node['url'] = 'https://fr.wikipedia.org/wiki/In_Deep'
df_edge = pd.DataFrame(data['links'])
df_edge.drop(columns=['value'],axis=1,inplace=True)

In [5]:
df_node.head()

,name,url
0,Myriel,https://fr.wikipedia.org/wiki/In_Deep
1,Napoleon,https://fr.wikipedia.org/wiki/In_Deep
2,Mlle.Baptistine,https://fr.wikipedia.org/wiki/In_Deep
3,Mme.Magloire,https://fr.wikipedia.org/wiki/In_Deep
4,CountessdeLo,https://fr.wikipedia.org/wiki/In_Deep


In [6]:
df_edge.head()

,source,target
0,1,0
1,2,0
2,3,0
3,3,2
4,4,0


In [ ]:
# df_node = pd.read_csv(DF_NODE_PATH)
# df_edge = pd.read_csv(EDGE_PATH)

In [7]:
labels = df_node['name'].tolist()
color_node = [0.0 for i in range(len(labels))]
texts_to_show = [None for i in range(len(labels))]

# Create plotting items
title, annotations, textbox_query, selector, query_answer, g = create_plot_items(df_node,df_edge,labels,color_node,texts_to_show)

In [8]:
#bert_embedder_spectral, bert_embedder_mean, node2vec_embed, df_node = load_models()

In [9]:
# On a new query, compute the predictions and color the nodes accordingly
def response(change):
    current_selector = selector.value
    query = textbox_query.value
#     preds = make_prediction(query, method, bert_embedder_spectral, bert_embedder_mean, node2vec_embed, df_node)
    preds = {10:1,11:0.8,12:0.6,13:0.4,14:0.3}
    dict_colors = compute_color(preds)
   
    html_text = create_text(list(dict_colors.keys()),df_node)
    query_answer.value = html_text
    
    for k,v in dict_colors.items():
        color_node[k] = v
        texts_to_show[k] = labels[k]
        
    with g.batch_update():
        g.data[1].marker.color = color_node
        g.data[1].text = texts_to_show

textbox_query.observe(response, names="value")
selector.observe(response, names="value")

# Open url when clicking on node
def update_point(trace, points, selector):
    if len(points.point_inds) != 0:
        url = g.data[1].customdata[points.point_inds[0]]
        webbrowser.open_new_tab(url)

g.data[1].on_click(update_point)

In [10]:
container = widgets.HBox(children=[textbox_query, selector])
widgets.VBox([title, annotations, container, query_answer, g])